In [1]:
import pandas as pd
import edsnlp
from spacy import displacy

import spacy
import datetime
import re


In [37]:
# - Listes expressions a eviter
list_embolie_no_MTEV = "|".join(
    [
        " graisseuse",
        " m.?tastatique",
        " septique",
        " bact.?rienne",
        " gazeuse",
    ]
)

# Expression Reguliere : RegEx

regex_MTEV = dict(
    # MTEV
    EP=[
        rf"(?i)embolie(?!.{{0,10}}{list_embolie_no_MTEV})",
        r"(?i)t(|h)romb(us|i\W).{0,50}(pulmonaire|lobaire)",
        r"(?i)infarctus.{0,10}pulmonaire",
        r"\WE\.P\.\W",
        r"\WEP\W",
    ],
)


def build_nlp_pipe_MTEV():
    
    """
    Builds the nlp pipe needed to extract MTEV.

    Returns
    -------
        nlp : object spacy.blank
            EDS-NLP pipeline.
    """
    
    nlp = edsnlp.blank("eds")

    # sentencizer component
    nlp.add_pipe("eds.sentences")
    nlp.add_pipe("eds.normalizer")

    # Matcher component
    nlp.add_pipe(
        "eds.matcher",
        config=dict(regex=regex_MTEV, ignore_excluded=True),
    )

    # Qualifiers
    nlp.add_pipe("eds.negation")  # Negation component
    nlp.add_pipe("eds.hypothesis")  # Speculation pipeline
    nlp.add_pipe("eds.family")  # Family context detection
    nlp.add_pipe(
        "eds.sections"
    )  # Section where entity is quoted

    # Extracting dates
    nlp.add_pipe("eds.dates")

    return nlp



def search_MTEV(note_mtev_df):
    """
    Performs the actual textual search of MTEV entities.

    Parameters
    ----------
        note_mtev_df : DataFrame Pandas
            Notes of CR containing at least the following columns :
            - `person_id`,
            - `visit_occurrence_id`,
            - `note_id`,
            - `note_class_source_value`,
            - `note_datetime`,
            - `note_text`.

    Returns
    -------
        ents_df : DataFrame Pandas
            Containing the MTEV found in the text and the potential associated
            date.
    """
    nlp = build_nlp_pipe_MTEV()

    docs = edsnlp.data.from_pandas(
        note_mtev_df,
        converter="omop",
        doc_attributes=[
            "person_id",
            "visit_occurrence_id",
            "note_class_source_value",
            "note_datetime",
        ],
    )
    # Add the pipeline to operations that will be run
    docs = docs.map_pipeline(nlp)
    # Distribution of the operations of lazy collection on multiple workers
    docs = docs.set_processing(backend="multiprocessing", show_progress=True)
    ents_df = docs.to_pandas(converter=pick_results)

    return ents_df


def separe_paragraphe(ent, nlp):
    """
    This function returns as entity's sentence the paragraph containing the
    entity.

    Parameters
    ----------
        ent : spacy.Span
            Entity found in a doc by nlp pipes.

    Returns
    -------
        tokenizer : spacy.Span
            Paragraph (text surrounded by "\n\n") containing the entity in the
            doc.
    """
    liste = pd.Series(re.split(r"\n\W*\n", ent.sent.text))
    txt = liste[liste.str.contains(ent.text)].reset_index().squeeze()[0]

    debut = ent.sent.text.find(txt)  # re.search(txt, ent.sent.text).start()
    fin = debut + len(txt)  # re.search(txt, ent.sent.text).end()

    tokenizer = spacy.tokenizer.Tokenizer(nlp.vocab)

    return tokenizer(ent.sent.text[debut:fin])






def pick_results(doc):
    """
    This function provides the entities that must be collected by the nlp
    process.

    Parameters
    ----------
        doc : spacy.Doc
            A container with the annotated text.

    Returns
    -------
        list[dict]
            Shape to stock the results for each entity found.
    """
    return [
        {
            "person_id": e.doc._.person_id,
            "visit_occurrence_id": e.doc._.visit_occurrence_id,
            "note_id": e.doc._.note_id,
            "note_class_source_value": e.doc._.note_class_source_value,
            "lexical_variant": e.text,
            "label": e.label_,
            "negation": e._.negation,
            "hypothesis": e._.hypothesis,
            "family": e._.family,
            "sentence":e.sent.text,
            "section": e._.section,
            
            
        }
        for e in doc.ents
        if doc.ents
    ]


In [15]:
df_CR=pd.read_excel("../Stage_sadjia/CR_.xlsx")

In [16]:
df_CR

,person_id,note_title,note_class_source_value,note_datetime,note_text
0,1,CR Urgences SAT SAU ACCUEIL (UF),INCONNU,2021-05-08 20:20:00,Adressé par MDS Faidherbe pour sensation de dy...
1,2,CR Urgences BCT URGENCES ADULTE,INCONNU,2021-09-10 00:00:00,Compte-rendu de passage aux Urgences\nAL.Score...


## Type de comptes rendus

In [17]:
df_CR.note_text[0]

'Adressé par MDS Faidherbe pour sensation de dyspnée avec frissons ce jour.\\nTachycardie à l\'IAO. Avis coord : PEC rapide.\\n- Infirme moteur cérébral\\nAsthme\\nDyspnée\\nTA : 111/77 mmHg FC : 104 Puls/min Temp : 37.5 °C FR : 16 /min\\nSpO2 : 99 % EVA : 0 CGS : 15\\nBU : 08/05/2021 21:32\\nLeucocytes : Ø Nitrites : Ø Protéine : + PH : 7.0 Erythrocytes : Ø Densité : 1.0 Cétone : Ø\\nGlucose : Ø\\nPA bras Dt : 156/98 mmHg FC : 145 /mn Temp : 37.8 °C FR : 20 /mn\\nSpO2 : 96% air\\nGCS : 15 (Y: 4/V: 5/M :6)\\nEVA : 0\\nMonsieur VANCIU JANNON\\n9 RUE LEBOUI\\n75014 PARIS 14\\n2/5CR Urgences SAT SAU ACCUEIL (UF), Imprimé le 09/05/2021 02:51\\nPat.: vanciu JANNON | M | 30/01/1996 | 8010143495 | 761236428\\nTraitements habituels\\nMode de vie\\nAnamnèse\\nExamen clinique initial\\nEvolution\\nExamens complémentaires\\nVentoline\\nBaclofène\\nIMC\\nvit en foyer se déplace en fauteuil electrique\\nPatient de 25 ans IMC es déplacant en fauteuil\\naderssé par maison de santé Faidherbe pour dysp

In [18]:
df_CR.note_text[1]

'Compte-rendu de passage aux Urgences\\nAL.Score de tri : Score 2\\nMotif de tri : Dyspnée / insuffisance respiratoire\\nParamètres vitaux, à l\\\'accueil :\\nPA :155/70 mmHg FC : 84 /mn Temp : 36.5 °C FR : 30 /mn\\nSpO2 : 88% air\\nGCS : 15 (Y: 4/V: 5/M :6)\\nParamètres vitaux, dernière prise :\\nTA : 152/76 mmHg Temp : 36.4 °C FR : 22 /min SpO2 : 92 %\\nEVA : 0 CGS : 15\\n8\\nAntécédents\\n.37 Allergies :\\n- Pas d\\\'allergieconnue\\n59 Antécédents médicaux :\\n- AOMI stentée\\n3/06/1934 | 8006963017 | 2800038564\\nBPCO post tabagique\\nHTA\\nNodulte pulmonaire suspect en cours exploration\\nTraitements habituels\\ncf ordonnance scannée\\nMode de vie\\nvit au domicile avec sa femme\\npas d\\\'aide au domicile\\ntrouble cognitif débutant\\nAnamnèse\\nDétresse respiratoire amené par SP, vue par AR au domicile\\nSat 88 => SAUV\\na eu 1 serie d\\\'aerosol\\nPatient présentant des troubles de la mémoire : ne sait pas ce qu\\\'il fait ici, ne sait pas son âge.\\nBronchite chronique à prio

## Mention positive d'Embolie Pulmonaire dans les CR_URGENCE

### 1ère méthode avec l'application des fonctions nlp

In [38]:
df_EP=search_MTEV(df_CR)

2024-05-23 11:35:51.786 | WARNING  | edsnlp.pipes.qualifiers.family.family:__init__:142 - You have requested that the pipeline use annotations provided by the `section` pipeline, but it was not set. Skipping that step.
2024-05-23 11:35:51.786 | WARNING  | edsnlp.pipes.misc.sections.sections:__init__:122 - The component Sections is still in Beta. Use at your own risks.
2it [00:00,  3.26it/s]


In [39]:
df_EP

,person_id,visit_occurrence_id,note_id,note_class_source_value,lexical_variant,label,negation,hypothesis,family,sentence,section
0,2,None,None,INCONNU,embolie,EP,False,False,False,nPas d\'argument pour une embolie pulmonaire j...,None
1,2,None,None,INCONNU,embolie,EP,False,False,False,nCONCLUSION :\nPas d\'embolie pulmonaire.\,None


## 2ème méthode: Extraction des patients avec embolie pulmonaire sans fonction

In [49]:
# Initialisation du pipeline NLP en dehors de la boucle
nlp = edsnlp.blank("eds")

regex_MTEV = {
    "EP": [
        r"(?i)embolie(?!.{0,10} graisseuse| m.?tastatique| septique| bact.?rienne| gazeuse)",
        r"(?i)t(|h)romb(us|i\W).{0,50}(pulmonaire|lobaire)",
        r"(?i)infarctus.{0,10}pulmonaire",
        r"\WE\.P\.\W",
        r"\WEP\W",
    ],
}

# Ajout des composants au pipeline
nlp.add_pipe("eds.matcher", config=dict(regex=regex_MTEV, ignore_excluded=True))
nlp.add_pipe("eds.sentences")
nlp.add_pipe("eds.negation")
nlp.add_pipe("eds.hypothesis")
nlp.add_pipe("eds.family")
nlp.add_pipe("eds.sections")

results = []
entities = []

# Traitement des textes du DataFrame
for i, text in enumerate(df_CR.note_text):
    doc = nlp(text)
    
    for ent in doc.ents:
        entity = {
            "id_patient": df_CR.person_id.iloc[i],
            "lexical_variant": ent.text,
            "label": ent.label_,
            "negation": ent._.negation,
            "hypothesis": ent._.hypothesis,
            "family": ent._.family,
            "sentence": ent.sent.text,
            "section": ent._.section
        }
        entities.append(entity)
        
        if not ent._.negation and not ent._.hypothesis:
            results.append(entity)

# Conversion des listes de résultats en DataFrame
df_EP = pd.DataFrame.from_records(results)
df_entities = pd.DataFrame.from_records(entities)



2024-05-23 11:41:15.974 | WARNING  | edsnlp.pipes.qualifiers.family.family:__init__:142 - You have requested that the pipeline use annotations provided by the `section` pipeline, but it was not set. Skipping that step.
2024-05-23 11:41:15.974 | WARNING  | edsnlp.pipes.misc.sections.sections:__init__:122 - The component Sections is still in Beta. Use at your own risks.
2024-05-23 11:41:15.995 | WARNING  | edsnlp.pipes.misc.sections.sections:__init__:150 - You should add pipe `eds.normalizer`


In [48]:
df_entities

,id_patient,lexical_variant,label,negation,hypothesis,family,sentence,section
0,1,embolie,EP,True,False,False,nTachycardie sans argument pour une embolie pu...,None
1,2,embolie,EP,False,False,False,nPas d\'argument pour une embolie pulmonaire j...,None
2,2,embolie,EP,False,False,False,nCONCLUSION :\nPas d\'embolie pulmonaire.\,None


In [50]:
df_EP

,id_patient,lexical_variant,label,negation,hypothesis,family,sentence,section
0,2,embolie,EP,False,False,False,nPas d\'argument pour une embolie pulmonaire j...,None
1,2,embolie,EP,False,False,False,nCONCLUSION :\nPas d\'embolie pulmonaire.\,None
